In [2]:
!nvidia-smi

Wed Oct 11 08:56:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:90:00.0 Off |                    0 |
| N/A   32C    P0    56W / 400W |  18396MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:

import os
os.system('pip install huggingface_hub accelerate bitsandbytes peft deep_translator')

os.system('pip install git+https://github.com/huggingface/transformers.git@19f0b7dd02c7ea7cbf86cc87fe00667470266722')
os.system('pip install git+https://github.com/huggingface/peft.git@99f792e8a3be001c55e71d148a7a8d25b931181f')
          
os.system('huggingface-cli login --token ')

In [3]:


import json
import transformers
from deep_translator import GoogleTranslator
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

def load_configuration(config_file):
    with open(config_file, 'r') as config_file:
        return json.load(config_file)

def load_model(model_id, peft_model_id, load_in_8bit, device_map):
    model = AutoModelForCausalLM.from_pretrained(model_id, device_map=device_map, load_in_8bit=load_in_8bit)
    model.load_adapter(peft_model_id)
    return model



def translate_text(text, model, tokenizer):
    prompt = f"<s> You are going to work as a professional translator, i will provide you the sentence in spanish and you will answer with the translation and after that the word FINISHED. Sentence: {text} [/INST]"
    
    text_generation_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer
    )

    sequences = text_generation_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=int(2.25*len(tokenizer.tokenize(prompt)))
    )

    output = sequences[0]['generated_text']
    return output, prompt

if __name__ == "__main__":
    
    config = load_configuration('config.json')
    model_id = config["model_id"]
    peft_model_id = "RikoteMaster/llama2_translation"
    load_in_8bit = config["load_in_8bit"]
    device_map = config["device_map"]
    
    model = load_model(model_id, peft_model_id, load_in_8bit, device_map)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    
    


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards:   0%|                                                                                                                                                                                                                                                                                                                                | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/storage.py:315: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.
  warnings.warn(message, UserWarning)
Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████

In [ ]:
import time

text = """Hola buenas, acabo de entrar a este foro porque quiero tener información de lo que me ocurre. Os cuento..Tengo 45 años y desde mi infancia he ido creciendo con ansiedad en la escuela y con inseguridades en la adolescencia que me impedían relacionarme correctamente siempre sintiéndome nervioso y ansioso por todo lo nuevo que me pasaba. No he sabido reconducir eso.Ya en la edad adulta estuve en un trabajo que me estresaba mucho, tenía mucha presión por parte del encargado, iba al trabajo cabreado, no tenía apetito casi, vaya... que estaba muy deprimido. Pasé muchos meses así hasta que un día me dió un ataque de ansiedad bastante grande con muchos espasmos involuntarios en brazos, cuello y cabeza y descontrolados. Me asusté y acudí a Urgencias, me dieron Diazepam y remitió.Ese fue mi primer gran susto.Otro gran susto fue que un día cualquiera comiendo se me quedó una bola de comida en la garganta y no pasaba ni para arriba ni para abajo, tenía los músculos de la garganta agarrotados y estuve unos segundos así hasta que se relajaron pero ya no pude volver a tragar bien.. cosas como sopas, guisos, cosas de cuchara, no he vuelto a ser el mismo que antes, perdí mucho peso estaba en los huesos.Sólo me calmaba el diazepan pero tengo tal preocupación por lo que me pasa y por la incertidumbre que tengo los músculos de la garganta agarrotados y no puedo deglutir bien la comida y claro después de 20 años tomando medicación no he vuelto a ser la misma persona. Me vió un psiquiatra y me mandó paroxetina más Diazepam pero no he ido a terapia.También de las mismas preocupaciones y pensamientos negativos mi cabeza a generado un transtorno obsesivo compulsivo de ir revisando cosas como grifos que estén cerrados, puerta de entrada de casa, etc...Que me está pasando?Será un problema neurológico?Podré llegar a ser una persona normal algún día?Que piensan ustedes?Gracias por leerlo."""
start_time = time.time()

texto, prompt = translate_text(text, model, tokenizer)
# Detiene el cronómetro
end_time = time.time()

# Calcula y muestra la duración
duration = end_time - start_time
print(f"El código tardó {duration} segundos en ejecutarse.")
print(texto)



In [2]:
texto

"<s> You are going to work as a professional translator, i will provide you the sentence in spanish and you will answer with the translation and after that the word FINISHED. Sentence: Hola buenas, acabo de entrar a este foro porque quiero tener información de lo que me ocurre. Os cuento..Tengo 45 años y desde mi infancia he ido creciendo con ansiedad en la escuela y con inseguridades en la adolescencia que me impedían relacionarme correctamente siempre sintiéndome nervioso y ansioso por todo lo nuevo que me pasaba. No he sabido reconducir eso.Ya en la edad adulta estuve en un trabajo que me estresaba mucho, tenía mucha presión por parte del encargado, iba al trabajo cabreado, no tenía apetito casi, vaya... que estaba muy deprimido. Pasé muchos meses así hasta que un día me dió un ataque de ansiedad bastante grande con muchos espasmos involuntarios en brazos, cuello y cabeza y descontrolados. Me asusté y acudí a Urgencias, me dieron Diazepam y remitió.Ese fue mi primer gran susto.Otro 

In [ ]:
    config = load_configuration('config.json')
    model_id = config["model_id"]
    peft_model_id = config["peft_model_id"]
    load_in_8bit = config["load_in_8bit"]
    device_map = config["device_map"]

    model = load_model(model_id, peft_model_id, load_in_8bit, device_map)
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # Leer el JSON con el texto a clasificar
    with open('input.json', 'r') as input_file:
        input_data = json.load(input_file)
        input_text = input_data.get("text", "")

    

    if input_text:
        translated_text, prompt, emotion = classify_emotion(input_text, model, tokenizer)


    result = {
        "original_text": input_text,
        "translated_text": translated_text,
        "prompt_text": prompt,
        "emotion": emotion
    }

    # Guardar el resultado en un archivo JSON de salida
    with open('output.json', 'w') as output_file:
        json.dump(result, output_file, indent=4)

## TRYING DIFFERENT MODEL TO MAKE TRANSLATION

In [5]:
article = """Muy buenas.Estoy bastante desesperada, ni la psiquiatra obtiene respuesta a lo que me ocurre ni encuentro testimonios en internet parecidos a los míos. Parece ser que a casi todas las personas con TDAH alguna medicación le ha hecho efecto en su atención -a bien, me refiero-, aunque tuvieran efectos secundarios.En mi caso, ya he probado Concerta y Elvanse en las dosis más altas registradas y ninguna de las dos medicaciones me han ayudado ni en la atención (soy estudiante y es lo que más necesito) ni en la hiperactividad-impulsividad. Ambas medicaciones me dieron efectos secundarios (como el apetito nulo) y con Concerta además tuve que parar porque me daba efectos depresivos fuertes. Precisamente el día que no tomaba Concerta, no tenía ningún efecto depresivo.En España creo que ya no hay más opciones de medicación. Sufro mucho con el tema de la concentración y la procrastinación de tareas infinita. Mis análisis de sangre y orina también están bien. He leído que si no me hacen efecto quizá es porque también tenga autismo, pero lo dudo bastante. Además a algunas personas autistas también le recetan medicación para TDAH y mejoran. Por favor, ¿alguien sabe por qué no me hacen ningún efecto positivo?Gracias de antemano."""

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", use_auth_token=True, src_lang="ron_Latn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M", use_auth_token=True)

inputs = tokenizer(article, return_tensors="pt")

/usr/local/lib/python3.8/dist-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torch/storage.py:315: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.
  warnings.warn(message, UserWarning)
/usr/local/lib/python3.8/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [8]:
import time

start_time = time.time()

translated_tokens = model.generate(
     **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["eng_Latn"]
 )
print(tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0])

end_time = time.time()

print(f"La duración de la traducción es de {-start_time+end_time}")


I am quite desperate, and I do not have any answers to what I am experiencing on the internet. It seems that almost all people with ADHD have some medication that has an effect on their attention - well, I mean, even if it has a strong side effect. In my case, I have already experienced a Concerta y Elvanse en las más altas dosis registradas y ninguna de las medicas me han ayudado ni en la atención (I am a student y es lo que más necesito) or en la hiperactividad-impulsividad.
La duración de la traducción es de 16.553450107574463
